In [15]:
from langchain_community.document_loaders import ImageCaptionLoader
from tempfile import NamedTemporaryFile

from langchain.agents import initialize_agent
from langchain.chat_models import ChatOllama
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

from langchain.tools import BaseTool
from transformers import BlipProcessor, BlipForConditionalGeneration, DetrImageProcessor, DetrForObjectDetection
from PIL import Image
import torch

from langchain.callbacks import StreamingStdOutCallbackHandler

import os
import requests
from bs4 import BeautifulSoup
from mimetypes import guess_extension
from urllib.parse import urljoin  

In [12]:
base_url = "http://127.0.0.1:8123/"  
output_folder = "./imgs"


if not os.path.exists(output_folder):
    os.makedirs(output_folder)

response = requests.get(base_url)
soup = BeautifulSoup(response.content, "html.parser")

img_tags = soup.find_all("img")

for i, img in enumerate(img_tags):
    if not img.has_attr('alt'):
        img_url = img.get("src")
        if img_url:
            full_img_url = urljoin(base_url, img_url)  
            img_response = requests.get(full_img_url, stream=True)
            img_content_type = img_response.headers['Content-Type']
            img_extension = guess_extension(img_content_type)
            img_name = f"{i}{img_extension}"
            img_path = os.path.join(output_folder, img_name)
            with open(img_path, "wb") as img_file:
                for chunk in img_response.iter_content(chunk_size=8192):
                    img_file.write(chunk)
            print(f"다운로드: {img_path}")

print("done!")


다운로드: ./imgs/0.jpg
다운로드: ./imgs/2.jpg
done!


In [22]:
import os

img_path = './imgs'

img_url_list = [os.path.join(img_path, f) for f in os.listdir(img_path) if os.path.isfile(os.path.join(img_path, f))]

for i in img_url_list:
  print(i)


./imgs/0.jpg
./imgs/2.jpg


In [27]:
from xml.dom.minidom import Document

def getImgCaption(img_url_list):
    loader = ImageCaptionLoader(images=img_url_list)
    list_docs = loader.load()
    for doc in list_docs:
        print(doc.page_content)

getImgCaption(img_url_list)

/home/ga111o/document/MarkDown/kwu-idea-lab/projects/add-alt-using-llm/env/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/ga111o/document/MarkDown/kwu-idea-lab/projects/add-alt-using-llm/env/lib/python3.11/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


an image of a girl with a sword and text that reads, i'ver, i
an image of a book with a fish on it [SEP]


In [11]:
for i in img_url_list:
  os.remove("imgs/"+i)

In [ ]:

model = ChatOllama(
    model = "llama3:8b",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)